### Importing Necessary Libraries

In [ ]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.io import read_image

### Setting up CUDA and TensorBoard

In [ ]:
# CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA available: {torch.cuda.is_available()}")

# TensorBoard
writer = SummaryWriter()
print(f"Writer: {writer}")

### Main Hyperparameters

In [9]:
image_base_size = 256
num_epochs = 2
batch_size = 10
learning_rate = 0.001

### Creating the Monkey Dataset and Loading in Training & Validation Data

In [ ]:
class MonkeyDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform):
        self.annotations_file = annotations_file
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.number_samples = 0

        # Adding all monkeys to a list for indexing
        self.monkeys = []
        for dirname, _, filenames in os.walk(self.img_dir):
            for filename in filenames:
                self.number_samples += 1
                self.monkeys.append((
                    os.path.join(dirname, filename),
                    filename
                ))

    def __getitem__(self, index):
        monkey_path, monkey_filename = self.monkeys[index]

        monkey = read_image(monkey_path)  # This already converts to a tensor

        # Get monkey label from filename
        label = int(monkey_filename[1:2])  # Works since n0 to n9 all 2 characters
        
        # Apply the transforms
        if self.transform:
            tmonkey = self.transform(monkey)
        
        return tmonkey, label

    def __len__(self):
        return self.number_samples

    def get_label_map(self):
        # List of labels corresponding to monkey type
        return {i:j.strip() for i, j in zip(range(0, 10), self.img_labels.iloc[:, 2])}

### Initializing Datasets and Dataloaders

In [ ]:
# Composition of transforms
composed = transforms.Compose([
    transforms.ConvertImageDtype(torch.float32),
    transforms.Resize([image_base_size, image_base_size])  # Monkey images are not all square
])

# Training and validation datasets (different from dataloaders)
# Datasets gets passed into a dataloader
training_data = MonkeyDataset(
    annotations_file = "./kaggle/input/10-monkey-species/monkey_labels.txt",
    img_dir = "./kaggle/input/10-monkey-species/training/training",
    transform = composed
)

validation_data = MonkeyDataset(
    annotations_file = "./kaggle/input/10-monkey-species/monkey_labels.txt",
    img_dir = "./kaggle/input/10-monkey-species/validation/validation",
    transform = composed
)

# Dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)

# Peeking into the dataloader to get an example batch (train_dataloader is an iterator)
monkey_batch, train_labels = next(iter(train_dataloader))

# Showing an example monkey
# https://stackoverflow.com/a/66641911
plt.imshow(monkey_batch[0].permute(1, 2, 0))
plt.show()

# Use CUDA
monkey_batch = monkey_batch.to(device)
train_labels = train_labels.to(device)

# [[[image_channel1, image_channel2, image_channel3], ...]], [label1, ...]
print(f"Batch shape: {monkey_batch.shape}")
print(f"Image shape: {monkey_batch[0].shape}")

# Add batch of images to TensorBoard
writer.add_images("Example Batch", monkey_batch)

### Defining the Convolutional Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self) -> None:
        # Note: https://discuss.pytorch.org/t/dataset-inheritance-does-not-require-super/92945/2
        super().__init__()

        # Classification learning
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        
        # Feature learning (regular NN part)
        self.full_con1 = nn.Linear(8*8*24, 1000)
        self.full_con2 = nn.Linear(1000, 100)
        self.full_con3 = nn.Linear(100, 10)
    
    def forward(self, x):
        # F.leaky_relu is a function, self.conv1 was from a class that called the function somewhere
        # Function takes in a tensor as input and outputs tensor
        s = self.pool(F.leaky_relu(self.conv1(x)))  
        s = self.pool(F.leaky_relu(self.conv2(s)))  
        s = self.pool(F.leaky_relu(self.conv3(s)))

        # Output of classification learning squashed down:  channels * image width * image height
        s = torch.reshape(s, [-1, 8*8*24])

        s = F.leaky_relu(self.full_con1(s))
        s = F.leaky_relu(self.full_con2(s))
        s = self.full_con3(s)  # The last activation function (softmax) is applied by loss function

        return s

# Use CUDA
model = CNN().to(device)

# Add graph to TensorBoard
writer.add_graph(model, input_to_model=monkey_batch)

print(model)

### Optimizing and Training the CNN

In [15]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

step = 0

# Training loop
for epoch in range(num_epochs):
    for iteration, (_monkey_batch, _train_labels) in enumerate(train_dataloader):
        # Use CUDA
        _monkey_batch = _monkey_batch.to(device)
        _train_labels = _train_labels.to(device)

        # Forward pass
        # Note: model.forward() is the same as model()
        pred = model.forward(_monkey_batch)  # Returns torch.Size([batch_size, 10])
        size_batch = pred.shape[1]
        print(size_batch)

        # Clearing accumulated gradients from optimizer
        optimizer.zero_grad()

        # Calculate loss and do backwards pass
        # Target is a value between 0 and C
        print(_train_labels)
        loss = loss_fn(pred, _train_labels)  # Notes for what gets passed here: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
        loss.backward()  # Calculates gradients for optimizer step function

        # Backpropagation
        optimizer.step()

        _, max_pred = torch.max(pred, 1)
        # acc = 
        print(a, _train_labels)

        print(f"Epoch:\t{epoch}\tIteration:\t{iteration}\tLoss:\t{loss}")
        writer.add_scalar("Loss/train", loss, i)
        writer.flush()

        step += 1

writer.close()



10
tensor([8, 7, 1, 2, 8, 2, 9, 7, 8], device='cuda:0')
tensor([3, 6, 9, 9, 3, 9, 9, 9, 9], device='cuda:0') tensor([8, 7, 1, 2, 8, 2, 9, 7, 8], device='cuda:0')
Epoch:	0	Iteration:	0	Loss:	2.2788784503936768
10
tensor([3, 0, 0, 6, 8, 1, 8, 6, 5], device='cuda:0')
tensor([3, 6, 9, 9, 3, 9, 9, 9, 9], device='cuda:0') tensor([3, 0, 0, 6, 8, 1, 8, 6, 5], device='cuda:0')
Epoch:	0	Iteration:	1	Loss:	2.007495403289795
10
tensor([9, 2, 2, 7, 2, 8, 6, 8, 9], device='cuda:0')
tensor([3, 6, 9, 9, 3, 9, 9, 9, 9], device='cuda:0') tensor([9, 2, 2, 7, 2, 8, 6, 8, 9], device='cuda:0')
Epoch:	0	Iteration:	2	Loss:	2.0097546577453613
10
tensor([0, 6, 3, 3, 9, 7, 5, 2, 3], device='cuda:0')
tensor([3, 6, 9, 9, 3, 9, 9, 9, 9], device='cuda:0') tensor([0, 6, 3, 3, 9, 7, 5, 2, 3], device='cuda:0')
Epoch:	0	Iteration:	3	Loss:	2.164885997772217


KeyboardInterrupt: 

### Evaluating model accuracy with actual dataset

In [ ]:
with torch.no_grad():
    tot = 0
    right = 0
    for monkey, label in validation_dataloader:  # Rename this to validation_dataloader later
        monkey = monkey.to(device)
        label = label.to(device)
        # print(monkey.size(), label) # Actual monkey, actual label

        guess = model(monkey)
        maxes, indicies = torch.max(guess, 1)
        
        right += sum(indicies == label)
        tot += indicies.shape[0]
        # print(indicies, label, torch.sum())

        # print(torch.sum)

print(f"Final Acc: {right/tot}")


### Exporting Trained Model

In [ ]:
print("WIP")

### Random Testing Scripts

In [ ]:
'''
e = pd.read_csv("./kaggle/input/10-monkey-species/monkey_labels.txt")
e.iloc[:,3]

tf = monkey_batch
# print(tf, type(tf), tf.size())

conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
pool = nn.MaxPool2d(kernel_size=3, stride=3)
conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3)
# self.pool2 = nn.MaxPool2d(kernel_size=3, stride=3)
conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3)
# self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
# End up with size of 30 for the regular neural network
full_con1 = nn.Linear(8*8*24, 100)
full_con2 = nn.Linear(100, 50)
full_con3 = nn.Linear(50, 10)

# This whole first part is classification learning
s = pool(F.leaky_relu(conv1(tf)))  # Function takes in a tensor as input
print(type(s), s.size())
s = pool(F.leaky_relu(conv2(s)))
print(type(s), s.size())
s = pool(F.leaky_relu(conv3(s)))
print(type(s), s.size())

# Output channels * image width * image height
# s.view(-1, 30*30*24).size()
s = torch.reshape(s, [-1, 8*8*24])
s.size()
# s = 

# This whole second part is feature learning
s = F.leaky_relu(full_con1(s))
print(type(s), s.size())
s = F.leaky_relu(full_con2(s))
print(type(s), s.size())
s = full_con3(s)
print(type(s), s.size())


print(model.base.parameters())
print(model.classifier.parameters())
for p in model.parameters():
    print(p)
'''